In [1]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [27]:
data = pd.read_csv("../../data/processed/data.csv",
                   dtype = {
                            "expensa_id": "object",
                            "unidad_id": "object",
                            "propietario_id": "object",
                            "inquilino_id": "object",
                            "consorcio_id": "object",
                            "expensa_padre_id": "object",
                            "unidad_prop": "object",
                            "consorcio_id": "object",
                            "usuario_creador_id": "object",
                            "concepto_id": "object",
                            "mes": "object",
                            "anio": "object",
                            "unidad_denominacion":"object",
                            "consorcio_nombre": "object",
                            "version":"object",
                            "concepto_id": "object",
                            "unidad_tipo": "object",
                            "modo_pago": "object",
                            "forma_prorrateo": "object",
                            "tipo_prorrateo": "object",
                            "metodo_pago": "object",                       
                    },
                    parse_dates = ["fecha_vencimiento_1", "fecha_vencimiento_2"])
data.fecha_pago = pd.to_datetime(data.fecha_pago, errors="coerce")
data["fecha_expensa"] = data.anio + '-' + data.mes + '-01'
data.fecha_expensa = pd.to_datetime(data.fecha_expensa)
data["fecha_expensa_anterior"] = data.fecha_expensa - pd.DateOffset(months=1)
data["impago"] = data.fecha_pago.isna()
data["pago_tardio_1"] = (data.fecha_pago > data.fecha_vencimiento_1) & (data.fecha_pago <= data.fecha_vencimiento_2)
data["pago_tardio_2"] = (data.fecha_pago > data.fecha_vencimiento_2)
data["pago_tardio"] = data.pago_tardio_1 | data.pago_tardio_2
data.loc[data.metodo_pago.isna(),"metodo_pago"] = "Impago"

In [3]:
data.shape

(264541, 33)

## Agregado de Tamaño de Consorcio

In [4]:
consorcios_raw = pd.read_csv("../../data/raw/expensas_consorcios.csv",
                         sep = ";",
                         na_values = "NULL",
                         dtype = {
                            "id": "object",
                        })

unidades_raw = pd.read_csv("../../data/raw/expensas_unidades.csv",
                         sep = ";",
                         na_values = "NULL",
                         dtype = {
                            "id": "object",
                            "consorcio_id":"object"
                        })

consorcios = consorcios_raw.loc[consorcios_raw.deleted == 0, ["id"]]
unidades = unidades_raw.loc[unidades_raw.consorcio_id.isin(consorcios.id),:]
unidades = unidades.loc[unidades.deleted == 0,:]
unidades_consorcio = pd.merge(consorcios,
                              unidades,
                              left_on="id",
                              right_on="consorcio_id",
                              suffixes = ("","_unidad"))
unidad_por_consorcio = unidades_consorcio.groupby("consorcio_id").count()["id_unidad"].reset_index()

In [5]:
unidad_por_consorcio["log_tamanio_consorcio"] = np.round(np.log(unidad_por_consorcio.id_unidad))
unidad_por_consorcio.loc[unidad_por_consorcio.log_tamanio_consorcio <=1, "log_tamanio_consorcio"] = 1
unidad_por_consorcio.loc[unidad_por_consorcio.log_tamanio_consorcio >=5, "log_tamanio_consorcio"] = 5
unidad_por_consorcio["tamanio_consorcio"] = unidad_por_consorcio.log_tamanio_consorcio.map(str)
tamanios_consorcios = unidad_por_consorcio[["consorcio_id", "tamanio_consorcio"]]

In [28]:
data = pd.merge(data,
                tamanios_consorcios,
                left_on="consorcio_id",
                right_on="consorcio_id",
                how="left")

In [7]:
data.shape

(264541, 34)

In [8]:
data.columns

Index(['expensa_id', 'unidad_id', 'propietario_id', 'inquilino_id',
       'fecha_vencimiento_1', 'fecha_vencimiento_2', 'monto', 'monto_total',
       'monto_parcial', 'expensa_padre_id', 'int_dia', 'int_mes', 'mes',
       'anio', 'unidad_denominacion', 'unidad_prop', 'consorcio_id',
       'consorcio_nombre', 'version', 'concepto_id', 'unidad_tipo',
       'modo_pago', 'forma_prorrateo', 'tipo_prorrateo', 'fecha_pago',
       'monto_pago', 'metodo_pago', 'fecha_expensa', 'fecha_expensa_anterior',
       'impago', 'pago_tardio_1', 'pago_tardio_2', 'pago_tardio',
       'tamanio_consorcio'],
      dtype='object')

In [9]:
data = data[['expensa_id', 'unidad_id', 'fecha_vencimiento_1', 'fecha_vencimiento_2', 'monto', 'monto_total', 'int_dia', 'int_mes', 'mes', 'unidad_denominacion', 'unidad_prop', 'consorcio_id',
       'consorcio_nombre', 'version', 'fecha_pago', 'metodo_pago', 'fecha_expensa', 'fecha_expensa_anterior',
       'impago', 'pago_tardio_1', 'pago_tardio_2', 'pago_tardio', 'tamanio_consorcio']]

In [10]:
data.isna().sum()

expensa_id                    0
unidad_id                     0
fecha_vencimiento_1           0
fecha_vencimiento_2           0
monto                         0
monto_total                   0
int_dia                       0
int_mes                       0
mes                           0
unidad_denominacion           0
unidad_prop                   0
consorcio_id                  0
consorcio_nombre              0
version                       0
fecha_pago                36500
metodo_pago                   0
fecha_expensa                 0
fecha_expensa_anterior        0
impago                        0
pago_tardio_1                 0
pago_tardio_2                 0
pago_tardio                   0
tamanio_consorcio             0
dtype: int64

## Cantidad de Impagos hasta la fecha

In [11]:
def get_cum_by_unidad(unidad_id):
    data_unidad = data.loc[data.unidad_id == unidad_id, ["unidad_id","fecha_expensa", "impago", "pago_tardio", "pago_tardio_1", "pago_tardio_2"]].sort_values("fecha_expensa")
    data_unidad["cum_impago"] = data_unidad.impago.cumsum()
    data_unidad["cum_pago_tardio"] = data_unidad.pago_tardio.cumsum()
    data_unidad["cum_pago_tardio_1"] = data_unidad.pago_tardio_1.cumsum()
    data_unidad["cum_pago_tardio_2"] = data_unidad.pago_tardio_2.cumsum()
    return data_unidad

In [13]:
dfs = list()
for unidad in data.unidad_id.unique():
    data_unidad = get_cum_by_unidad(unidad)
    dfs.append(data_unidad)

In [14]:
final = pd.concat(dfs)

In [85]:
final.shape

(264541, 10)

In [29]:
data = pd.merge(data,
                final[["unidad_id","fecha_expensa", "cum_impago", "cum_pago_tardio", "cum_pago_tardio_1", "cum_pago_tardio_2"]],
                left_on=["fecha_expensa_anterior","unidad_id"],
                right_on=["fecha_expensa", "unidad_id"],
                how = "left")

,expensa_id,unidad_id,fecha_vencimiento_1,fecha_vencimiento_2,monto,monto_total,int_dia,int_mes,mes,unidad_denominacion,...,impago,pago_tardio_1,pago_tardio_2,pago_tardio,tamanio_consorcio,fecha_expensa_y,cum_impago,cum_pago_tardio,cum_pago_tardio_1,cum_pago_tardio_2
0,2588,135,2014-06-10,2014-06-25,2940.63,122499.07,5.0,10.0,5,2 (B),...,False,False,True,True,4.0,NaT,NaN,NaN,NaN,NaN
1,3372,135,2014-08-10,2014-08-25,3948.45,122499.07,5.0,10.0,7,2 (B),...,False,False,True,True,4.0,NaT,NaN,NaN,NaN,NaN
2,7643,135,2014-11-10,2014-11-25,1669.52,80511.69,5.0,10.0,10,2 (B),...,False,False,False,False,4.0,NaT,NaN,NaN,NaN,NaN
3,8880,135,2014-12-10,2014-12-25,3123.80,148806.68,5.0,10.0,11,2 (B),...,False,False,False,False,4.0,2014-10-01,0.0,2.0,0.0,2.0
4,12923,135,2015-01-10,2015-01-25,2723.84,127312.34,5.0,10.0,12,2 (B),...,False,False,False,False,4.0,2014-11-01,0.0,2.0,0.0,2.0
5,14658,135,2015-02-10,2015-02-25,2379.66,112226.71,5.0,10.0,1,2 (B),...,False,False,False,False,4.0,2014-12-01,0.0,2.0,0.0,2.0
6,16270,135,2015-03-10,2015-03-25,1996.88,94292.58,5.0,10.0,2,2 (B),...,False,True,False,True,4.0,2015-01-01,0.0,2.0,0.0,2.0
7,18947,135,2015-04-10,2015-04-25,2295.75,108096.11,5.0,10.0,3,2 (B),...,False,False,True,True,4.0,2015-02-01,0.0,3.0,1.0,2.0
8,20472,135,2015-05-10,2015-05-25,2060.99,97701.72,5.0,10.0,4,2 (B),...,False,True,False,True,4.0,2015-03-01,0.0,4.0,1.0,3.0
9,22581,135,2015-06-10,2015-06-25,2639.60,123724.35,5.0,10.0,5,2 (B),...,False,False,False,False,4.0,2015-04-01,0.0,5.0,2.0,3.0


In [30]:
data["venc_1_dayofweek"] = data.fecha_vencimiento_1.dt.dayofweek.map(str)
data["venc_1_day"] = data.fecha_vencimiento_1.dt.day
data["venc_2_dayofweek"] = data.fecha_vencimiento_2.dt.dayofweek.map(str)
data["venc_2_day"] = data.fecha_vencimiento_2.dt.day
data["mes_vencimiento"] = data.fecha_vencimiento_1.dt.year.map(str) + '-' + data.fecha_vencimiento_1.dt.month.map(str) + '-01'
data.mes_vencimiento = pd.to_datetime(data.mes_vencimiento)
data["dia_inicio_mes_vencimiento"] = data.mes_vencimiento.dt.dayofweek

In [32]:
data_mes_anterior = data[["fecha_expensa_x", "unidad_id", "venc_1_dayofweek", "venc_1_day", "venc_2_dayofweek", "venc_2_day", "metodo_pago", "impago", "pago_tardio", "pago_tardio_1", "pago_tardio_2", "cum_impago", "cum_pago_tardio", "cum_pago_tardio_1", "cum_pago_tardio_2"]]

In [33]:
data = pd.merge(
    data,
    data_mes_anterior,
    left_on=["fecha_expensa_anterior","unidad_id"],
    right_on=["fecha_expensa_x", "unidad_id"],
    how = "left",
    suffixes = ("","_ant"))

In [34]:
data.to_csv("../../data/processed/data_2.csv", index=False)